<a href="https://colab.research.google.com/github/ayberkuslu/MLProject/blob/master/Copy_of_Water_Pump_2_Linear_SVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

train = r"/content/preprocessed_train.csv"
test = r"/content/preprocessed_test.csv"

train = pd.read_csv(train)
test = pd.read_csv(test)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             59400 non-null  float64
 1   days_since_recorded    59400 non-null  int64  
 2   funder                 59400 non-null  object 
 3   installer              59400 non-null  object 
 4   basin                  59400 non-null  object 
 5   population             59400 non-null  int64  
 6   public_meeting         59400 non-null  object 
 7   scheme_management      59400 non-null  object 
 8   permit                 59400 non-null  object 
 9   construction_year      59400 non-null  object 
 10  extraction_type_class  59400 non-null  object 
 11  payment_type           59400 non-null  object 
 12  water_quality          59400 non-null  object 
 13  quantity               59400 non-null  object 
 14  source_type            59400 non-null  object 
 15  so

In [ ]:
# Get dummy columns for the categorical columns and shuffle the data.

dummy_cols = ['funder', 'installer', 'basin', 'public_meeting', 'scheme_management', 'permit','construction_year', 'extraction_type_class', 'payment_type', 'water_quality','quantity', 'source_type', 'source_class', 'waterpoint_type','waterpoint_type_group']

train = pd.get_dummies(train, columns = dummy_cols)

train = train.sample(frac=1).reset_index(drop=True)

In [ ]:
train.shape

(59400, 94)

In [ ]:
test = pd.get_dummies(test, columns = dummy_cols)

In [ ]:
test.shape
test.head(2)

,amount_tsh,days_since_recorded,population,funder_danida,funder_gov,funder_hesawa,funder_other,funder_rwssp,funder_world_bank,installer_commu,installer_danida,installer_dwe,installer_gov,installer_other,installer_rwe,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,basin_Rufiji,basin_Ruvuma / Southern Coast,basin_Wami / Ruvu,public_meeting_False,public_meeting_True,public_meeting_Unknown,scheme_management_other,scheme_management_vwc,scheme_management_wtr_auth,scheme_management_wtr_brd,scheme_management_wua,scheme_management_wug,permit_False,permit_True,permit_Unknown,construction_year_00s,construction_year_10s,construction_year_60s,construction_year_70s,...,payment_type_on failure,payment_type_other,payment_type_per bucket,payment_type_unknown,water_quality_coloured,water_quality_fluoride,water_quality_fluoride abandoned,water_quality_milky,water_quality_salty,water_quality_salty abandoned,water_quality_soft,water_quality_unknown,quantity_dry,quantity_enough,quantity_insufficient,quantity_seasonal,quantity_unknown,source_type_borehole,source_type_dam,source_type_other,source_type_rainwater harvesting,source_type_river/lake,source_type_shallow well,source_type_spring,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
0,0.0,302,321,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,0.0,302,300,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
# Let's split the train set into train and validation sets. Also remove the target.

target = train.status_group
features = train.drop('status_group', axis=1)

X_train, X_val, y_train, y_val = train_test_split(features, target, train_size=0.8)

In [ ]:
# Both the train and test set are ready for modelling. Start off with a Linear Support Vector 
# Classification.
# http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html

def model(X_train, X_val, y_train, y_val, test):
    if __name__ == '__main__':

        pipe_svc = Pipeline([('scl', StandardScaler()),
                             ('clf', LinearSVC())])
    
        param_grid = {'clf__C':[0.001, 0.01, 0.1, 1.0],
                      'clf__class_weight':[None, 'balanced']}

        estimator = GridSearchCV(estimator=pipe_svc,
                                 param_grid=param_grid,
                                 n_jobs=-1)

        estimator.fit(X_train, y_train)

        best_params = estimator.best_params_
                                 
        validation_accuracy = estimator.score(X_val, y_val)
        print('Validation accuracy: ', validation_accuracy)
        print(best_params)

In [ ]:
model(X_train, X_val, y_train, y_val, test)
#Validation accuracy:  0.735101010101
#{'clf__class_weight': None, 'clf__C': 0.01}

Validation accuracy:  0.735101010101
{'clf__class_weight': None, 'clf__C': 0.01}


In [ ]:
# Get data necessary for submission.

submit_loc = r"/content/SubmissionFormat.csv"
test_id = pd.read_csv(submit_loc)
test_id.columns = ['idd', 'status_group']
test_id = test_id.idd

In [ ]:
def model_for_submission(features, target, test):
    if __name__ == '__main__':

        pipe_svc = Pipeline([('scl', StandardScaler()),
                             ('clf', LinearSVC())])
    
        param_grid = {'clf__C':[0.01],
                      'clf__class_weight':[None]}

        estimator = GridSearchCV(estimator=pipe_svc,
                                 param_grid=param_grid,
                                 n_jobs=-1, verbose=15)

        estimator.fit(features, target)        

        predictions = estimator.predict(test)

        data = {'ID': test_id, 'status_group': predictions}

        submit = pd.DataFrame(data=data)

        vals_to_replace = {2:'functional', 1:'functional needs repair',
                           0:'non functional'}

        submit.status_group = submit.status_group.replace(vals_to_replace)        

        submit.to_csv('Linear-results.csv', index=False)

In [ ]:
# Run model for submission.

model_for_submission(features, target, test)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.4min


In [ ]:
# The model scored 0.7318. Below are scores from other models I ran using less data and/or no
# scaling. 

#0.6873 without funder
#0.6750 with funder
#0.7101 with funder and scaling